<a href="https://colab.research.google.com/github/45xjh/PanoHead-colab/blob/main/PanoHead_custom_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%cd /content
!git clone -b dev1 https://github.com/camenduru/PanoHead

!apt -y install -qq aria2
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/PanoHead/resolve/main/ablation-trigridD-1-025000.pkl -d /content/PanoHead/models -o ablation-trigridD-1-025000.pkl
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/PanoHead/resolve/main/baseline-easy-khair-025000.pkl -d /content/PanoHead/models -o baseline-easy-khair-025000.pkl
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/PanoHead/resolve/main/easy-khair-180-gpc0.8-trans10-025000.pkl -d /content/PanoHead/models -o easy-khair-180-gpc0.8-trans10-025000.pkl

!pip install imgui glfw pyspng mrcfile ninja plyfile trimesh onnxruntime onnx

%cd /content
!git clone -b dev https://github.com/camenduru/3DDFA_V2
%cd /content/3DDFA_V2
!sh ./build.sh

!cp -rf /content/PanoHead/3DDFA_V2_cropping/test /content/3DDFA_V2
!cp /content/PanoHead/3DDFA_V2_cropping/dlib_kps.py /content/3DDFA_V2
!cp /content/PanoHead/3DDFA_V2_cropping/recrop_images.py /content/3DDFA_V2

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/shape_predictor_68_face_landmarks/resolve/main/shape_predictor_68_face_landmarks.dat -d /content/3DDFA_V2 -o shape_predictor_68_face_landmarks.dat

!mkdir /content/in /content/stage /content/output

/content
Cloning into 'PanoHead'...
remote: Enumerating objects: 155, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 155 (delta 9), reused 6 (delta 6), pack-reused 145
Receiving objects: 100% (155/155), 29.40 MiB | 14.69 MiB/s, done.
Resolving deltas: 100% (45/45), done.
The following additional packages will be installed:
  libaria2-0 libc-ares2
The following NEW packages will be installed:
  aria2 libaria2-0 libc-ares2
0 upgraded, 3 newly installed, 0 to remove and 8 not upgraded.
Need to get 1,513 kB of archives.
After this operation, 5,441 kB of additional disk space will be used.
Selecting previously unselected package libc-ares2:amd64.
(Reading database ... 129824 files and directories currently installed.)
Preparing to unpack .../libc-ares2_1.18.1-1ubuntu0.22.04.2_amd64.deb ...
Unpacking libc-ares2:amd64 (1.18.1-1ubuntu0.22.04.2) ...
Selecting previously unselected package libaria2-0:amd64.
Preparing to unpack ...

In [2]:
%cd /content/3DDFA_V2
!rm -rf /content/stage/* /content/3DDFA_V2/crop_samples/img/* /content/3DDFA_V2/test/original/* /content/output/*
!cp /content/in/* /content/3DDFA_V2/test/original

!python dlib_kps.py
!python recrop_images.py -i data.pkl -j dataset.json

!cp -rf /content/3DDFA_V2/crop_samples/img/* /content/stage

# %cd /content/PanoHead
# !./gen_pti_script.sh

/content/3DDFA_V2
100% 1/1 [00:02<00:00,  2.19s/it]
results: 1


In [3]:
%cd /content/PanoHead
!python projector_withseg.py --num-steps=300 --num-steps-pti=300 --outdir=/content/output --target_img=/content/stage --network=/content/PanoHead/models/easy-khair-180-gpc0.8-trans10-025000.pkl --idx 0

/content/PanoHead
Loading networks from "/content/PanoHead/models/easy-khair-180-gpc0.8-trans10-025000.pkl"...
projecting: [0] /content/stage/test.jpg
camera matrix: torch.Size([1, 25])
Computing W midpoint and stddev using 10000 samples...
Setting up PyTorch plugin "bias_act_plugin"... Done.
Setting up PyTorch plugin "upfirdn2d_plugin"... Done.
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1501: UserWarning: operator() profile_node %106 : int = prim::profile_ivalue(%104)
 does not have profile information (Triggered internally at ../third_party/nvfuser/csrc/graph_fuser.cpp:104.)
  return forward_call(*args, **kwargs)
step:    1/300 mse: 0.83 perc: 0.56 w_norm: 0.00  noise: 0.00 
step:    2/300 mse: 0.73 perc: 0.55 w_norm: 0.00  noise: 0.00 
step:    3/300 mse: 0.59 perc: 0.56 w_norm: 0.00  noise: 0.00 
step:    4/300 mse: 0.49 perc: 0.51 w_norm: 0.00  noise: 0.00 
step:    5/300 mse: 0.53 perc: 0.52 w_norm: 0.00  noise: 0.00 
step:    6/300 mse: 0.39 perc: 0.44 w_

In [4]:
%cd /content/PanoHead
!python gen_videos_proj_withseg.py --output=/content/output/easy-khair-180-gpc0.8-trans10-025000.pkl/0/PTI_render/pre.mp4 --latent=/content/output/easy-khair-180-gpc0.8-trans10-025000.pkl/0/projected_w.npz --trunc 0.7 --network /content/PanoHead/models/easy-khair-180-gpc0.8-trans10-025000.pkl --cfg Head

/content/PanoHead
Loading networks from "/content/PanoHead/models/easy-khair-180-gpc0.8-trans10-025000.pkl"...
Setting up PyTorch plugin "bias_act_plugin"... Done.
Setting up PyTorch plugin "upfirdn2d_plugin"... Done.
100% 240/240 [00:23<00:00, 10.08it/s]


In [8]:
%cd /content/PanoHead
!python gen_videos_proj_withseg.py --output=/content/output/easy-khair-180-gpc0.8-trans10-025000.pkl/0/PTI_render/post.mp4 --latent=/content/output/easy-khair-180-gpc0.8-trans10-025000.pkl/0/projected_w.npz --trunc 0.7 --network /content/output/easy-khair-180-gpc0.8-trans10-025000.pkl/0/fintuned_generator.pkl --cfg Head

/content/PanoHead
Loading networks from "/content/output/easy-khair-180-gpc0.8-trans10-025000.pkl/0/fintuned_generator.pkl"...

Aborted!


In [14]:
%cd /content/PanoHead
!python gen_samples.py --outdir=/content/output/easy-khair-180-gpc0.8-trans10-025000.pkl/0/test --trunc=0.7 --shapes=true --seeds=0 \
    --network models/easy-khair-180-gpc0.8-trans10-025000.pkl

/content/PanoHead
Loading networks from "models/easy-khair-180-gpc0.8-trans10-025000.pkl"...
Generating image for seed 0 (0/1) ...
Setting up PyTorch plugin "bias_act_plugin"... Done.
Setting up PyTorch plugin "upfirdn2d_plugin"... Done.
135000000it [00:07, 17797485.46it/s]                   
